In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [3]:
states = vq.get_initial_state()
events = vq.get_event_table()

In [7]:
aux = vq.get_round_gamestate(5)
pd.concat(aux)

,Round,EventIndex,Time,Player Side,Loadout Value,Weapon Price,Shield,Remaining Creds,Controller,Duelist,Initiator,Sentinel,Has Operator
0,5,0,0,attack,24000,14500,250,15900,1,2,1,1,0
1,5,0,0,defense,23250,16600,200,750,1,1,2,1,1
0,5,1,72891,attack,24000,14500,250,15900,1,2,1,1,0
1,5,1,72891,defense,18550,13700,150,650,1,1,2,0,1
0,5,2,73216,attack,18500,11600,200,13450,1,2,1,0,0
1,5,2,73216,defense,18550,13700,150,650,1,1,2,0,1
0,5,3,80084,attack,13900,8700,150,8850,1,2,0,0,0
1,5,3,80084,defense,18550,13700,150,650,1,1,2,0,1
0,5,4,80351,attack,13900,8700,150,8850,1,2,0,0,0
1,5,4,80351,defense,12900,8700,125,450,1,0,2,0,0


In [4]:
mad = vq.generate_match_alive_dict()
chosen_round = 1
chosen_event = 2
alive_list = mad[chosen_round][chosen_event]
query = states[(states["Round"] == chosen_round)]
row_pot = []
for row_index in range(len(query)):
    row = states.iloc[row_index].copy()
    player_name = row["Player Name"]
    status = row["Status"]
    is_alive = alive_list[player_name]
    row["Status"] = "Alive" if is_alive == 1 else "Dead"
    row_pot.append(row)
g = pd.DataFrame(row_pot)
g = g[g["Status"] == "Alive"]
agg_dict = {"Remaining Creds": "sum", "Loadout Value": "sum", "Shield": "sum", "Weapon Price": "sum",
            "Controller": "sum", "Duelist": "sum", "Initiator": "sum", "Sentinel": "sum", "Has Operator": "sum"}
g.groupby(["Player Side"]).agg(agg_dict).reset_index()
#g.apply(lambda x: x[x["Status"] == "Alive"]).

,Player Side,Remaining Creds,Loadout Value,Shield,Weapon Price,Controller,Duelist,Initiator,Sentinel,Has Operator
0,attack,250,3950,50,1500,1,2,1,1,0
1,defense,0,2200,50,500,1,0,1,1,0


# Json Style

In [10]:
chosen_round = 5
round_states = states[(states["Round"] == chosen_round)]
round_states = round_states.sort_values(by=["Player Side"])
round_events = events[(events["Round"] == chosen_round)]
alive_info = {item: "alive" for item in round_states["Player Name"].unique()}
agg_dict = {"Remaining Creds": "sum", "Loadout Value": "sum", "Shield": "sum", "Weapon Price": "sum",
            "Controller": "sum", "Duelist": "sum", "Initiator": "sum", "Sentinel": "sum", "Has Operator": "sum"}
gamestate_pot = [0]
for row_index in range(len(round_events)):
    row = round_events.iloc[row_index]
    event_type = row["EventType"]
    victim_name = row["VictimName"]
    if event_type == "kill":
        alive_info[victim_name] = "dead"
    elif event_type == "revival":
        alive_info[victim_name] = "alive"

    state_pot = []
    for state_index in range(len(round_states)):
        state = round_states.iloc[state_index].copy()
        player_name = state["Player Name"]
        state["Status"] = alive_info[player_name]
        state_pot.append(state)

    alive_df = pd.DataFrame(state_pot)
    alive_df = alive_df[alive_df["Status"] == "alive"]
    alive_df = alive_df.groupby(["Player Side"]).agg(agg_dict).reset_index()
    gamestate_pot.append(alive_df)

gamestate_pot[1]

,Player Side,Remaining Creds,Loadout Value,Shield,Weapon Price,Controller,Duelist,Initiator,Sentinel,Has Operator
0,attack,15900,24000,250,14500,1,2,1,1,0
1,defense,650,18550,150,13700,1,1,2,0,1


In [14]:
round_states

,Match ID,Round ID,Round,Player ID,Player Name,Status,Team ID,Player Side,Remaining Creds,Loadout Value,...,Weapon Name,Weapon Price,Utility Value,Agent Name,Starting Side,Controller,Duelist,Initiator,Sentinel,Has Operator
43,43621,695328,5,1877,frz,alive,53,attack,1450,4800,...,Vandal,2900,4800,Reyna,attack,0,1,0,0,False
44,43621,695328,5,3105,gtnziN,alive,53,attack,4250,4600,...,Vandal,2900,4600,Jett,attack,0,1,0,0,False
47,43621,695328,5,2895,Saadhak,alive,53,attack,2450,5500,...,Phantom,2900,5500,Sage,attack,0,0,0,1,False
48,43621,695328,5,2294,Sacy,alive,53,attack,4600,4600,...,Phantom,2900,4600,Sova,attack,0,0,1,0,False
49,43621,695328,5,194,sutecas,alive,53,attack,3150,4500,...,Vandal,2900,4500,Viper,attack,1,0,0,0,False
40,43621,695328,5,1843,ade,alive,444,defense,200,3700,...,Phantom,2900,3700,Viper,defense,1,0,0,0,False
41,43621,695328,5,1075,Bazzi,alive,444,defense,0,4600,...,Vandal,2900,4600,Sova,defense,0,0,1,0,False
42,43621,695328,5,1824,Fisker,alive,444,defense,200,5650,...,Operator,5000,5650,Jett,defense,0,1,0,0,True
45,43621,695328,5,876,Munchkin,alive,444,defense,250,4600,...,Vandal,2900,4600,KAY/O,defense,0,0,1,0,False
46,43621,695328,5,2388,neth,alive,444,defense,100,4700,...,Phantom,2900,4700,Sage,defense,0,0,0,1,False
